In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [27]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [28]:
encoded[:100]

array([ 6, 77,  0, 21, 59, 75, 32, 67, 49, 82, 82, 82, 22,  0, 21, 21, 23,
       67, 10,  0, 37, 24, 50, 24, 75, 48, 67,  0, 32, 75, 67,  0, 50, 50,
       67,  0, 50, 24, 79, 75, 51, 67, 75, 76, 75, 32, 23, 67, 78, 47, 77,
        0, 21, 21, 23, 67, 10,  0, 37, 24, 50, 23, 67, 24, 48, 67, 78, 47,
       77,  0, 21, 21, 23, 67, 24, 47, 67, 24, 59, 48, 67, 31, 72, 47, 82,
       72,  0, 23, 20, 82, 82, 12, 76, 75, 32, 23, 59, 77, 24, 47])

In [7]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [8]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [54]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [55]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [53]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 6 77  0 21 59 75 32 67 49 82]
 [67 47 31 72 67 50  0 48 59 75]
 [48 20 67 73 77 75 67 72 24 10]
 [78  0 32 32 75 50 71 67 27 32]
 [32 67 48 24 64 75 67  0 47 64]
 [67 59 75 48 31 32 31 57 17 17]
 [67 69 47 64 67 47 31 59 24 30]
 [18 31 72 47 67  0 50 72  0 23]
 [31 32 48 59 67 31 10 67  0 50]
 [78 59 75 67 72 77 75 47 71 67]]

y
 [[77  0 21 59 75 32 67 49 82 82]
 [47 31 72 67 50  0 48 59 75 64]
 [20 67 73 77 75 67 72 24 10 75]
 [ 0 32 32 75 50 71 67 27 32 24]
 [67 48 24 64 75 67  0 47 64 67]
 [59 75 48 31 32 31 57 17 17 47]
 [69 47 64 67 47 31 59 24 30 24]
 [31 72 47 67  0 50 72  0 23 48]
 [32 48 59 67 31 10 67  0 50 50]
 [59 75 67 72 77 75 47 71 67 31]]


In [56]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

No GPU available, training on CPU; consider making n_epochs very small.


In [57]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [58]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [59]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [60]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2702... Val Loss: 3.2186
Epoch: 1/20... Step: 20... Loss: 3.1606... Val Loss: 3.1402
Epoch: 1/20... Step: 30... Loss: 3.1467... Val Loss: 3.1246
Epoch: 1/20... Step: 40... Loss: 3.1160... Val Loss: 3.1196
Epoch: 1/20... Step: 50... Loss: 3.1456... Val Loss: 3.1180
Epoch: 1/20... Step: 60... Loss: 3.1173... Val Loss: 3.1157
Epoch: 1/20... Step: 70... Loss: 3.1084... Val Loss: 3.1139
Epoch: 1/20... Step: 80... Loss: 3.1198... Val Loss: 3.1089
Epoch: 1/20... Step: 90... Loss: 3.1129... Val Loss: 3.0963
Epoch: 1/20... Step: 100... Loss: 3.0790... Val Loss: 3.0664
Epoch: 1/20... Step: 110... Loss: 3.0318... Val Loss: 3.0033
Epoch: 1/20... Step: 120... Loss: 2.8890... Val Loss: 2.8797
Epoch: 1/20... Step: 130... Loss: 2.8306... Val Loss: 2.8124
Epoch: 2/20... Step: 140... Loss: 2.7172... Val Loss: 2.6677
Epoch: 2/20... Step: 150... Loss: 2.6258... Val Loss: 2.5760
Epoch: 2/20... Step: 160... Loss: 2.5646... Val Loss: 2.5211
Epoch: 2/20... Step: 170... Loss:

Epoch: 10/20... Step: 1350... Loss: 1.3979... Val Loss: 1.4293
Epoch: 10/20... Step: 1360... Loss: 1.4088... Val Loss: 1.4295
Epoch: 10/20... Step: 1370... Loss: 1.3929... Val Loss: 1.4282
Epoch: 10/20... Step: 1380... Loss: 1.4290... Val Loss: 1.4228
Epoch: 10/20... Step: 1390... Loss: 1.4470... Val Loss: 1.4207
Epoch: 11/20... Step: 1400... Loss: 1.4450... Val Loss: 1.4217
Epoch: 11/20... Step: 1410... Loss: 1.4466... Val Loss: 1.4181
Epoch: 11/20... Step: 1420... Loss: 1.4517... Val Loss: 1.4114
Epoch: 11/20... Step: 1430... Loss: 1.4088... Val Loss: 1.4141
Epoch: 11/20... Step: 1440... Loss: 1.4322... Val Loss: 1.4116
Epoch: 11/20... Step: 1450... Loss: 1.3559... Val Loss: 1.4114
Epoch: 11/20... Step: 1460... Loss: 1.3821... Val Loss: 1.4107
Epoch: 11/20... Step: 1470... Loss: 1.3847... Val Loss: 1.4065
Epoch: 11/20... Step: 1480... Loss: 1.4053... Val Loss: 1.4042
Epoch: 11/20... Step: 1490... Loss: 1.3911... Val Loss: 1.4040
Epoch: 11/20... Step: 1500... Loss: 1.3840... Val Loss:

Epoch: 20/20... Step: 2660... Loss: 1.2481... Val Loss: 1.3003
Epoch: 20/20... Step: 2670... Loss: 1.2589... Val Loss: 1.2993
Epoch: 20/20... Step: 2680... Loss: 1.2496... Val Loss: 1.2946
Epoch: 20/20... Step: 2690... Loss: 1.2402... Val Loss: 1.2974
Epoch: 20/20... Step: 2700... Loss: 1.2415... Val Loss: 1.2952
Epoch: 20/20... Step: 2710... Loss: 1.2224... Val Loss: 1.3007
Epoch: 20/20... Step: 2720... Loss: 1.2217... Val Loss: 1.2977
Epoch: 20/20... Step: 2730... Loss: 1.2134... Val Loss: 1.2957
Epoch: 20/20... Step: 2740... Loss: 1.2077... Val Loss: 1.2958
Epoch: 20/20... Step: 2750... Loss: 1.2243... Val Loss: 1.2969
Epoch: 20/20... Step: 2760... Loss: 1.2148... Val Loss: 1.2954
Epoch: 20/20... Step: 2770... Loss: 1.2509... Val Loss: 1.2945
Epoch: 20/20... Step: 2780... Loss: 1.2695... Val Loss: 1.2941


In [61]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [62]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [63]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [64]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna, and the step was taking his husband
triumphed, and with a cord and was strange to her shrook and her excitement, and still made up his husband, there came up, tried to see a close. She went to her..

So offer a sort of man would be a luth said. He said to her has to be distanceful for the pretented thing
to her and there was
it and was trempling. Sengey sometours than she wanted to say a milurate things of minutes of hose that she was successful from this most persons, but his bottem to him in the strange for the same time so answer there was a great does of her come, he was so and coachman and himself that there warn now he had already did not
know what to be a glance, where what was ansorible so fine, and with the most sense of most booking face and danser and himself with a single woman who was anxieting to do the more than that in this mind and all out that he was talking of the subject to her supper,
but since there was to say anything that they were
silent. He handed them
t

In [65]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [66]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said, as she saw than the subject all this should not care to be
taken away,
and a sort of people
trying to say to her to the peasants,
but was telling him. The fins sound of the prince was to class a steps as she had been that his chief the carriage of the
second attention, they should say himself, and she was at the same, things as it was so too the most freelation.



Chapter 22


Alexey Alexandrovitch saw she thinking of her and shout, and had such a soleth of those care, and an acquaintance which had
started to subture to the states,
and a smile of anything is still that she
came up to them, and she stopped and taking a calse that said, as
though a little
sole of arranging he saw this sight of the
result that he wanted to be seen as he was. He saw
it to his brushes.

"I'm a strengt of their confession of the music, but that I said a musing about intellite, but I am all the marshal or
some of
all of the society of the way, the poatis, it's all this. I don't see him to tak